In [8]:
##### IMPORT ALL PACKAGES #####
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandasql import sqldf
%matplotlib inline
pysqldf = lambda q: sqldf(q, globals())

##### IMPORT ALL DATA #####
df_bom_movie_gross = pd.read_csv('data/bom.movie_gross.csv') # PRIMARY
df_imdb_name_basics = pd.read_csv('data/imdb.name.basics.csv')
df_imdb_title_akas = pd.read_csv('data/imdb.title.akas.csv')
df_imdb_title_basics = pd.read_csv('data/imdb.title.basics.csv') # PRIMARY
df_imdb_title_crew = pd.read_csv('data/imdb.title.crew.csv')
df_imdb_title_principals = pd.read_csv('data/imdb.title.principals.csv')
df_imdb_title_ratings = pd.read_csv('data/imdb.title.ratings.csv') # PRIMARY
df_rt_movie_info = pd.read_csv('data/rt.movie_info.tsv', delimiter = '\t')
df_rt_reviews = pd.read_csv('data/rt.reviews.tsv', delimiter = '\t', encoding= 'unicode_escape')
df_tmdb_movies = pd.read_csv('data/tmdb.movies.csv').drop('Unnamed: 0', axis=1)
df_tn_movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')

In [9]:
GENRES_OG = ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']
GENRES_FULL = [
     'genre_action','genre_adult','genre_adventure','genre_animation','genre_biography','genre_comedy','genre_crime',
     'genre_documentary','genre_drama','genre_family','genre_fantasy','genre_gameshow','genre_history','genre_horror',
     'genre_music','genre_musical','genre_mystery','genre_news','genre_reality_tv','genre_romance','genre_sci_fi',
     'genre_short','genre_sport','genre_talkshow','genre_thriller','genre_war','genre_western']
GENRE_CONVERSION = dict(zip(GENRES_FULL, GENRES_OG))

In [10]:
##### COMBINE AND CLEAN FINANCIAL DATA #####
df_money = df_tn_movie_budgets.loc[:, ['movie', 'production_budget', 'domestic_gross','worldwide_gross']].copy()
production_budget = [float(x.replace('$', '').replace(',','')) for x in df_money.production_budget] 
domestic_gross = [float(x.replace('$', '').replace(',','')) for x in df_money.domestic_gross]
worldwide_gross = [float(x.replace('$', '').replace(',','')) for x in df_money.worldwide_gross]
df_money = pd.DataFrame([production_budget, domestic_gross, worldwide_gross]).transpose()
df_money.columns = ['production_budget', 'domestic_gross', 'worldwide_gross']
df_money['profit'] = df_money.worldwide_gross - df_money.production_budget
df_money['movie'] = df_tn_movie_budgets.movie
df_money = df_money[['movie','production_budget', 'domestic_gross', 'worldwide_gross', 'profit']]
df_money = df_money.sort_values(by='movie').reset_index(drop=True)
df_genres = df_imdb_title_basics[df_imdb_title_basics['genres'].isna() == False].copy()
for category in GENRES_OG:
    df_genres[category] = [category in x for x in df_genres['genres']]
df_genres.drop(['start_year', 'runtime_minutes', 'genres', 'original_title'], axis=1, inplace=True)

In [11]:
df = pysqldf(''' SELECT * FROM df_imdb_title_basics basics
LEFT JOIN df_genres genres USING (tconst)
LEFT JOIN df_money money ON basics.primary_title == money.movie
LEFT JOIN df_tmdb_movies tmdb ON basics.primary_title == tmdb.original_title;''')

df = df[[
       'tconst', 'primary_title', 'original_title', 'start_year', 'runtime_minutes', 
       'production_budget', 'domestic_gross', 'worldwide_gross', 'profit',
       'genre_ids', 'id', 'original_language', 'popularity',
       'release_date', 'title', 'vote_average', 'vote_count',
       'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']]
df.columns = ['tconst', 'primary_title', 'DELETE1', 'original_title',
       'DELETE2', 'start_year', 'runtime_minutes', 'production_budget',
       'domestic_gross', 'worldwide_gross', 'profit', 'genre_ids', 'id_tmdb',
       'original_language', 'popularity', 'release_date', 'title',
       'vote_average', 'vote_count', 'genre_action', 'genre_adult', 'genre_adventure',
       'genre_animation', 'genre_biography', 'genre_comedy', 'genre_crime', 'genre_documentary', 'genre_drama',
       'genre_family', 'genre_fantasy', 'genre_gameshow', 'genre_history', 'genre_horror', 'genre_music',
       'genre_musical', 'genre_mystery', 'genre_news', 'genre_reality_tv', 'genre_romance', 'genre_sci_fi',
       'genre_short', 'genre_sport', 'genre_talkshow', 'genre_thriller', 'genre_war', 'genre_western']
df.drop(['DELETE1', 'DELETE2'], axis=1, inplace=True)

In [12]:
MINIMUM_VOTES = 10 

profit_median = []
for g in GENRES_FULL:
    profit_median.append(df.loc[(df[g].isna()==False)&(df.profit.isna()==False)&(df[g]==True), 'profit'].median())

profit_stddev = [] 
for g in GENRES_FULL:
    profit_stddev.append(df.loc[(df[g].isna()==False)&(df.profit.isna()==False)&(df[g]==True), 'profit'].std())
    
popularity_median = []
for g in GENRES_FULL:
    popularity_median.append(df.loc[(df[g].isna()==False)&(df.popularity.isna()==False)&(df[g]==True),'popularity'].median())

voteavg_median = []
for g in GENRES_FULL:
    voteavg_median.append(df.loc[(df[g].isna()==False)&(df.vote_average.isna()==False)&(df[g]==True)&
                                  (df.vote_count>= MINIMUM_VOTES),'vote_average'].median())                  
film_ct = []
for g in GENRES_FULL:
    film_ct.append(df.loc[(df[g].isna()==False)&(df[g]==True),g].sum())


In [13]:
df_genres = pd.DataFrame(GENRES_FULL, columns = ['genre'])
df_genres['profit_median'] = profit_median
#df_genres['profit_stddev'] = profit_stddev#[x/10**7 for x in profit_stddev]
df_genres['popularity_median'] = popularity_median
df_genres['voteavg_median'] = voteavg_median
df_genres['film_ct'] = film_ct

QUANTILE_MIN = 0.4
quantiles = pd.DataFrame([[df_genres['profit_median'].quantile(QUANTILE_MIN), 
                           #df_genres['profit_stddev'].quantile(1-QUANTILE_MIN), 
                           df_genres['popularity_median'].quantile(QUANTILE_MIN), 
                           df_genres['voteavg_median'].quantile(QUANTILE_MIN),
                           df_genres['film_ct'].quantile(QUANTILE_MIN)]],
                         columns = ['profit_median','popularity_median', 'voteavg_median', 'film_ct'])

In [14]:
df_narrowed = df_genres.copy()

for g in df_narrowed.genre:
    for q in quantiles:
        df_narrowed[q] = df_narrowed[q].loc[df_narrowed[q] >= quantiles[q][0]]
        
df_narrowed = df_narrowed.dropna().reset_index(drop=True)
df_narrowed

,genre,profit_median,popularity_median,voteavg_median,film_ct
0,genre_adventure,58702781.5,4.2825,6.1,6704.0
1,genre_biography,9403192.0,1.8620,6.8,8966.0
2,genre_comedy,15852177.0,2.0915,6.0,25855.0
3,genre_romance,11057990.0,2.2120,6.1,9698.0
